# Setup Jmavsim and ROS
Follow the link to install PX4, ROS, and MAVROS. Install ROS-Desktop full version to avoid any error because of uninstalled library package.
https://docs.px4.io/main/en/dev_setup/dev_env_linux_ubuntu.html#ros-gazebo-classic


After installing everything, change the directory to `PX4-Autopilot` and start the jmavsim simulator by the following command:

`make px4_sitl_default jmavsim`

 Basics commands in the PX4-Shell are:
 
`commander takeoff` for taking off from the current coordinate<br>
`commander land` for landing at the current coordinate <br>
`commander mode offboard` for enabling the offboard command (MATLAB/Python/C++)<br>

Type `help` in the shell to explore the other commands.<br>
One useful command that might come handy is the `mavlink`, using `mavlink status`, one can find out the udp port and the mode(Normal-publishing at nominal frequecy, suitable for wifi transmission, Onboard - publishing at high frequency, suitable for an oboard computer)

## Jmavsim settings

Change the following parameter using `param set <name of the parameter>`, type `param` in the shell to understand how it works:

To enable offboard mode to 7 or check the qgroundcontrol for more functionalities:

`COM_RCL_EXCEPT`

For no RC and datalink error, change the following parameters to:

`NAV_RCL_ACT` 0 `COM_RC_IN_MODE` 1 

In PX4 shell, if you want to start a new mavlink enabled port then type the following coomand in the shell, this will give you a connection through udp port with baud rate 921600, one can know about other parameters of the mavlink command by typing `mavlink` in shell.

`mavlink start -m onboard -u 14555 -o 14551 -r 1000000 -t 127.0.0.1 -b 921600`

For increasing the stream rate of the sensor measurements, type the following command in PX4 shell:

`mavlink stream -u 14555 -s HIGHRES_IMU -r 350`

`mavlink stream -u 14555 -s LOCAL_POSITION_NED -r 100`

`mavlink stream -u 14555 -s ATTITUDE -r 350`

`mavlink stream -u 14555 -s ATTITUDE_QUATERNION -r 350`

OR hardcode these values in mavlink_main.cpp in `case MAVLINK_MODE_ONBOARD` part of the code and then do `make clean` and remake the px4_sitl to let it use the changed parameters.

## Pyulog error

if error occur while installing python2 pyulog packaage, then go to requirements.txt file in ~/mypx4/Firmware/Tools/setup and change pyulog>=0.5.0 to pyulog==0.7.0

## libgazebo9 error

If the following error shows: 

```python
The following packages have unmet dependencies:
 gazebo9 : Depends: libgazebo9 (= 9.19.0-2~focal) but it is not going to be installed
           Depends: gazebo9-common (>= 9.15.0) but it is not going to be installed
 libgazebo9-dev : Depends: libgazebo9 (= 9.19.0-2~focal) but it is not going to be installed
                  Depends: gazebo9-common (>= 9.15.0) but it is not going to be installed
                  Depends: gazebo9-plugin-base (= 9.19.0-2~focal) but it is not going to be installed
E: Unable to correct problems, you have held broken packages.
```

Then, type the following commands in terminal:

`sudo apt-get update`

`sudo apt-get install libgazebo9`

## java.awt.AWTError: Assistive Technology not found: org.GNOME.Accessibility.AtkWrapper

First fallback to older version of java i.e., version 8. To do that, type the following command - 

`sudo update-alternatives --config java`

and then select `java-8-openjdk`. Then type the following command to remove the previous file built for the other java version by:

`rm -rf Tools/jMAVSim/out`

If java 8 isn't installed in the system, then type the following commands isntead:

`sudo apt install openjdk-8-jdk`

`sudo update-alternatives --config java # choose 8`

`rm -rf Tools/jMAVSim/out`




``` python
Exception in thread "main" java.lang.reflect.InvocationTargetException
at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
at java.lang.reflect.Method.invoke(Method.java:498)
at org.eclipse.jdt.internal.jarinjarloader.JarRsrcLoader.main(JarRsrcLoader.java:58)
Caused by: java.awt.AWTError: Assistive Technology not found: org.GNOME.Accessibility.AtkWrapper
at java.awt.Toolkit.loadAssistiveTechnologies(Toolkit.java:807)
at java.awt.Toolkit.getDefaultToolkit(Toolkit.java:886)
at java.awt.Window.getToolkit(Window.java:1358)
at java.awt.Window.init(Window.java:506)
at java.awt.Window.(Window.java:537)
at java.awt.Frame.(Frame.java:420)
at java.awt.Frame.(Frame.java:385)
at javax.swing.JFrame.(JFrame.java:189)
at me.drton.jmavsim.Visualizer3D.(Visualizer3D.java:104)
at me.drton.jmavsim.Simulator.(Simulator.java:157)
at me.drton.jmavsim.Simulator.main(Simulator.java:678)
```
If you see this error, then edit the accessibility.properties file:

`sudo gedit /etc/java-8-openjdk/accessibility.properties`

and comment out the line indicated below:

`#assistive_technologies=org.GNOME.Acessibility.AtkWrapper`

# python environment change in matlab

To change it to python 2.7, type the foollowing command in matlab terminal:

`pyenv('Version','/usr/bin/python2.7')`


# Custom ROS Messages in Simulink
For setting thrust and body angular rate command signals, one should use mavros_msgs/setpoint_raw/attitude topic to send the command through simulink via ROS toolbox.
This message isn't available in simulink by default, therefore we will create a custom message using https://www.mathworks.com/help/ros/ug/create-custom-messages-from-ros-package.html

Copy .msg file of AttitudeTarget for thrust and body angular rate command signals from the following directory in your ros computer:

`roscd mavros_msgs/msg`

Paste your .msg file into msg folder as shown below:

mavros_msgs/<br>
├── mavros_msgs/<br>
│ ├── msg/<br>
│ │ └── AttitudeTarget.msg<br>
│ └── package.xml<br>

Then in Matlab command window, define folderpath as
`folderpath = '/home/ugupta62/Quad_simulation/mavros_msgs/mavros_msgs';` or wherever you want

Example of custom msgs folder structure -


Follow the rest of the steps given in https://www.mathworks.com/help/ros/ug/create-custom-messages-from-ros-package.html


Note: rosgenmsg(folderpath) function uses python 2.7 -64bit version to build the custom messages, hence one should change the path variable in windows(https://geek-university.com/python/add-python-to-the-windows-path/) or ubuntu. Make sure that your MATLAB is using python 2.7 by checking with `pyenv` command in matlab window. 

<!--Before running rosgenmsg, make sure that you have installed CMake to generate build system and visual studio C++ 2017 to compile the files.
Note: ROS toolbox can generate custom messages and code only using Visual studio C++ 2017 compiler.-->

## rosgenmsg cmake error
If the following error shows up:
``` python
Could not find CMake in your system. Please
install CMake version 3.15.5 or higher and
rerun the command.
```

Then check cmake version in matlab command window by the following command:

`!cmake --version`

This will give you an error of:

`cmake: /opt/matlab/r2021a/sys/os/glnxa64/libstdc++.so.6: version 'GLIBCXX_3.4.26' not found (required by cmake)`

This is caused because the version of this particular library packaged with MATLAB is older than the versions that come with newer Linux operating systems, which causes compatibility issues. So the workaround is renaming the libstdc++.so.6 library file so that MATLAB cannot find it and is forced to use the system's version of the library. This file is located in `matlabroot/sys/os/glnxa64` . Renaming it to `libstdc++.so.6.old` should suffice. (where "matlabroot" is root installation directory of MATLAB, generally it's `/opt/matlab/r2021a/`)

# Docker build for unbuntu 18.04

Create a Dockerfile by copying the below commands using a texteditor, then save the file with the name dockerfile_nvidia_ros without any extension.

`FROM nvidia/cudagl:11.1.1-base-ubuntu18.04`

`# Minimal setup
RUN apt-get update \
 && apt-get install -y locales lsb-release
ARG DEBIAN_FRONTEND=noninteractive
RUN dpkg-reconfigure locales`

`# Install ROS Melodic
RUN sh -c 'echo "deb http://packages.ros.org/ros/ubuntu$(lsb_release -sc) main" > /etc/apt/sources.list.d/ros-latest.list'
RUN apt-key adv --keyserver 'hkp://keyserver.ubuntu.com:80' --recv-key C1CF6E31E6BADE8868B172B4F42ED6FBAB17C654
RUN apt-get update \
 && apt-get install -y --no-install-recommends ros-melodic-desktop-full
RUN apt-get install -y --no-install-recommends python-rosdep
RUN rosdep init \
 && rosdep fix-permissions \
 && rosdep update
RUN echo "source /opt/ros/melodic/setup.bash" >> ~/.bashrc


# Docker commands

If docker gets stuck in downloading something over net, then check for the VPN. If it's connected then disconnect it.

1. To start a new container:

    `docker run -i -t <build_docker(here it's nvidia_ros)>`'
    

2. To ckeck the list of all containers even exited ones:

    `docker container list -a`


3. To re-start the exited container

    `docker restart <container_name (Can see from the above coomand)>`


4. To attach the continer with the input/output devices keyboard i,e, to open a new terminal

    `docker attach <container_name>`
  

# Airsim installation

Run the following commands:

`pip install msgpack-rpc-python`

`pip install airsim`

Download any one environemnt from https://github.com/Microsoft/AirSim/releases

Download the zip file for the environment you want, extract it somewhere and run .sh file present in the LinuxNoEditor folder.

# Airsim window resizing
The INI file, GameUserSettings.ini is located under environmental name for example, if you have AirSimNH environment and then the INI would be at AirSimNH\AirSimNH\Saved\Config\LinuxNoEditor folder.

Try changing this to the following:

```
[/Script/Engine.GameUserSettings]
 bUseVSync=False
 ResolutionSizeX=800
 ResolutionSizeY=600
 bUseDesktopResolutionForFullscreen=False
 FullscreenMode=2
 LastConfirmedFullscreenMode=2
 Version=5
 ```


# Airsim PX4 in SITL

Follow the link
https://microsoft.github.io/AirSim/px4_sitl/

Use following command to build and start PX4 firmware in SITL mode: 

`make px4_sitl_default none_iris`

# Propellors data by UIUC

https://m-selig.ae.illinois.edu/props/propDB.html

https://m-selig.ae.illinois.edu/props/volume-1/propDB-volume-1.html

# How to use Airsim

Go to UnrealEngine installation folder in Airsim and start Unreal by running`./Engine/Binaries/Linux/UE4Editor`.

When Unreal Engine prompts for opening or creating project, select Browse and choose `AirSim/Unreal/Environments/Blocks (or your custom Unreal project)`.

Alternatively, the project file can be passed as a commandline argument. For Blocks: `./Engine/Binaries/Linux/UE4Editor <AirSim_path>/Unreal/Environments/Blocks/Blocks.uproject`

If you get prompts to convert project, look for More Options or Convert-In-Place option. If you get prompted to build, choose Yes. If you get prompted to disable AirSim plugin, choose No.

After Unreal Editor loads, press Play button.

# Run PX4 shell

move to px4/PX4-Autopilot and then run `make px4_sitl none_iris` in terminal


# Settings.json file Airsim

```python
{
    "SettingsVersion": 1.2,
    "SimMode": "Multirotor",
    "ClockType": "SteppableClock",
    "PawnPaths": {
      "DefaultQuadrotor": {"PawnBP": "Class'/AirSim/BluePrints/BP_Flying_Ocotorotor_coax.BP_Flying_Ocotorotor_coax_C' "}
    },
    "Vehicles": {
        "PX4": {
            "Model": "Octocopter_Coax",
            "VehicleType": "PX4Multirotor",
            "UseSerial": false,
            "LockStep": true,
            "UseTcp": true,
            "TcpPort": 4560,
            "ControlPortLocal": 14540,
            "ControlPortRemote": 14580,
            "Sensors":{
                "Barometer":{
                    "SensorType": 1,
                    "Enabled": true,
                    "PressureFactorSigma": 0.0001825
                }
            },
            "Parameters": {
                "NAV_RCL_ACT": 0,
                "NAV_DLL_ACT": 0,
                "COM_OBL_ACT": 1,
                "LPE_LAT": 47.641468,
                "LPE_LON": -122.140165
            }
        }
    }
  }
```

# Crazyswarm

`pip install vispy`

`pip install pyusb`



# 3DR X8-M Octocopter

It's an old hardware, so it will not work with the latest stable release of PX4 due to the reason mentioned on the given [website](https://ardupilot.org/copter/docs/common-limited-firmware.html#common-limited-firmware). Workaround is to download the px4_fmu-v2_default.px4 of stable release [v1.11.1](https://github.com/PX4/PX4-Autopilot/releases/tag/v1.11.1) of PX4 and flash using Qgroundcontrol by slecting the custom firmware installtion in the drop down menu.

# Enabling UART on Raspberry Pi4 and Rock Pi 4b+ with ubuntu

## For Raspberry Pi
Default UART serial port on Raspberry Pi is /dev/ttyAMA0. In order to enable the UART on raspi open the file `sudo nano /boot/firmware/config.txt` and add the following lines:

`enable_uart=1`

`dtoverlay=disable-bt` - this will disbale the bluetooth, which is by defualt enabled on this port

and in `sudo nano /boot/firmware/cmdline.txt` the file, find `console=serial0,115200` and remove that part to disable the login shell on serial interface..

If want to check the baudrate speed on the port, type the following command (also the following dev is the serial port of raspberry pi)- 

`stty -F /dev/ttyAMA0`

Not the exact solution, but some directions to work with 

https://docs.px4.io/v1.devmerge/en/flight_controller/raspberry_pi_pilotpi_rpios.html

https://askubuntu.com/questions/1254376/enable-uart-communication-on-pi4-ubuntu-20-04

## For RockPi 4b+

Open file `sudo nano /boot/hw_intfc.conf` and enable UART4 by chnaging it's mode to on. Default port for UART2 is /dev/ttyS2 and for UART4 is /dev/ttyS4.(Check GPIO pins for specific UARTs)

Sometimes it happens, that after enabling the pixhawk or other devices can't access or open the serial port due to permission reason and show such error `Cannot open /dev/ttyS4: Permission denied`. 
The tty devices belong to the "dialout" group, being not a member of this group one will be denied access to /dev/ttyS4, so add yourself to that group.

First check if you are a member of that group:

`groups ${USER}`

If not then add yourself using -

`sudo gpasswd --add ${USER} dialout`

https://askubuntu.com/questions/210177/serial-port-terminal-cannot-open-dev-ttys0-permission-denied

# netplan file configuration

<!-- Since NetworkManager and systemd-networkd clash with each other, so stop and disable the NetworkManager by following commands in terminal:

`sudo systemctl stop NetworkManager
 sudo systemctl disable NetworkManager
 sudo reboot -->
`

Now, open netplan file by `sudo nano /etc/netplan/50-cloud-init.yaml`
Do the following changes: (caution keep indentation in mind)
```
network:
    ethernets:
        eth0:
            dhcp4: true
            optional: true
    version: 2
    #renderer: NetworkManager #enabling it makes a user to run netplan apply everytime after bootup
    wifis:
        wlan0:
            optional: true
            access-points:
              "TP-Link_C624_5G":
                password: "24737395"
            dhcp4: no
            addresses: [192.168.10.114/24]   # 192.168.10.110 represents static ip address
            gateway4: 192.168.10.3       # this gateway can be found on wifi router settings page -
            #nameservers:                 # type 192.168.10.3 on any browser, then admin, admin as credentials
             # addresses: [192.168.10.3,8.8.8.8]  
# The last two lines "addresses" are for DNS server settings, can be left alone since they are not used in IFL network
        wlan1:
            access-points:
              "TP-Link_C624_5G":
                password: "24737395"
            dhcp4: no
            addresses: [192.168.10.113/24]
# Don't define same gateway again 

```

Then, 

`sudo netplan apply`

# Vicon bridge

Use [Vicon_bridge](https://github.com/ethz-asl/vicon_bridge) repo for getting the vicon data to ROS.

Set the launch as follows:
```
<?xml version="1.0"?>

<launch>

  <!-- IP and port on Vicon Windows PC -->
  <arg name="datastream_hostport" default="192.168.10.2:801"/>
  <!-- Fixed vicon world frame -->
  <arg name="vicon_world_frame" default="/vicon/world"/>

	<node pkg="vicon_bridge" type="vicon_bridge" name="vicon" output="screen" >
		<param name="stream_mode" value="ServerPush"/>
    <param name="datastream_hostport" value="$(arg datastream_hostport)"/>
		<param name="tf_ref_frame_id" value="$(arg vicon_world_frame)"/>
	</node>

</launch>
```

# Converting USB flash drive to RAM

Type `sudo fdisk -l` in order to see the usb drive name on computer

In my case it's `dev/sda1`. Now follow the steps given on https://10pm.ca/using-a-usb-flash-drive-as-ram-in-linux-mintubuntu/


1. Plug it in. If it automounts, unmount it.
2. Type: sudo mkswap /dev/sda1 (or whatever your device path is)
3. Then type: sudo swapon -p 32767 /dev/sda1 (the 32767 makes it a higher priority and will thus be used before the hard drive swap drive) 

Plus, how to convert it back, see the GUI answer, i.e, second part on https://unix.stackexchange.com/questions/224156/how-to-safely-turn-off-swap-permanently-and-reclaim-the-space-on-debian-jessie

# Setting up Ubuntu server image on Rock Pi

Download the ubuntu server image from https://wiki.radxa.com/Rockpi4/downloads. 

Rock PI4B+ doesn't come with an installed OS, so flash the downloaded image on memory card with [balena etcher](https://www.balena.io/etcher/) and insert it in the memory card slot of the board. Before booting it up, connect HDMI cable

Download or transfer the above image xz file to rock pi. Then unzip it using -

`unxz file.xz`

Then type the following dd command to flash the image on eemc:

`sudo dd if=rockpi-4b-ubuntu-focal-server-arm64-20220311-0254-gpt.img of=/dev/mmcblk1`

Power off your Rock PI 4, remove the μSD and power on again. The system will now boot from the eMMC module. 



# How to compile cudy_wifi driver on rockpi

Since the rockpi has old kernel i.e., 4.4.154-116-rockchip-g86a614bc15b3, thus it will show up a compilation error of gcc-9, therefore in order to compile the driver, we need to shift to a previous version of gcc i.e, gcc-8. TO do that install gcc-8

`sudo apt install gcc-8`

`sudo apt install gcc++8`

Then choose the gcc-8 by update-alternative command-

`sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-8 8`

`sudo update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-8 8`

`sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-9 9`

`sudo update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-9 9`

Then choose gcc -8 and gcc+. 

Then compile your driver happily.